# Lab 5: Band math with Python

In lecture 5, we learned about highlighting specific features from the original bands of remote sensing image with multispectral transform. A very common category of multispectral transform is spectral indices, which when it comes to operation, is band math. In radio metric correction, since we need to modify the value of each pixels, we also need to apply certain functions to different bands, where band match is also needed. We will practice using band math for spectral indices calculation and DN matching in this lab. 




## Spectral Indices calculation
The following two code blocks accomplish the same purpose, calculate NDVI from a Landsat-8 image that the band is already stacked.  

### Python Functions
However, the second code block wraps things up with different functions. In Python, functions are blocks of reusable code that perform a specific task. A function typically takes some input, performs a series of operations, and then returns an output (or not).   
  

When you organize your Python code into functions, it is often referred to as "modularizing" the code. Modularization means breaking down a large piece of code into smaller, self-contained units (functions, classes, or modules), each with a specific responsibility. This practice improves the code's readability, maintainability, and reusability. **It is always a good habit to modulize your code!**  

[Optional readings on python functions](https://www.youtube.com/watch?v=89cGQjB5R4M)



### Numpy
Image data are stored in the format of multi-dimentional arrays in python. Numpy is one of the most powerful libraries to work with arrays for numerical computing.   

It provides support for handling large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.   

For example, it can directly calculate statistics of a numpy array with a built-in function like "Numpy_array.mean(), Numpy_array.sum()", you can also directly perform element-wise mathematical operations to numpy arrays such as "Numpy_array_1 + Numpy_array_2" or " Numpy_array * 2"

In the example code provided below, this line of code is used to calculate NDVI:  

**ndvi = np.where((nir_band + red_band) == 0., 0, (nir_band - red_band) / (nir_band + red_band))**  

When calculating the spectral indices, we need to be mindful that remote sensing images are suually surrounded by no-data values (explained in Lab 3), which are often 0. Since spectral indices are ratios, there can not be zero values in denominator or error will occure.

Here we use np.where(condition, a, b) function to handle potential division by zero errors.
in np.where, if condition returns True, then that pixel will be assigned value a, otherwise, that pixel will be assigned value b.

So the line of code provided here means: for each pixel, if "(nir_band + red_band) == 0." is true, then assign value 0 to this pixel, otherwise, the pixel value would be "(nir_band - red_band) / (nir_band + red_band)".  

[Optional readings on numpy](https://www.w3schools.com/python/numpy/numpy_intro.asp)


In [4]:
from osgeo import gdal
import numpy as np

# Input 8-band stacked GeoTIFF file
input_file = "Stacked_8bands_1.TIF"

# Open the input file and read in the original file parameters
dataset = gdal.Open(input_file)
x_size = dataset.RasterXSize
y_size = dataset.RasterYSize
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
data_type = dataset.GetRasterBand(1).DataType
no_data_value = dataset.GetRasterBand(1).GetNoDataValue()


# Read the NIR (band 5) and Red (band 4) bands as numpy arrays
nir_band = dataset.GetRasterBand(5).ReadAsArray().astype(np.float32)
red_band = dataset.GetRasterBand(4).ReadAsArray().astype(np.float32)

# Calculate NDVI using the formula (NIR - Red) / (NIR + Red)
# Remote sensing images are usually surrounded by no-data values
ndvi = np.where((nir_band + red_band) == 0., 0, (nir_band - red_band) / (nir_band + red_band))

# Define output NDVI file path
output_path = "Stacked_8bands_1_ndvi.TIF"

# Create the output file with one band for NDVI
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(output_path, x_size, y_size, 1, gdal.GDT_Float32)

# Set the projection and geotransform of the output file
output_dataset.SetProjection(projection)
output_dataset.SetGeoTransform(geotransform)

# Write the NDVI array to the output file
output_band = output_dataset.GetRasterBand(1)
output_band.WriteArray(ndvi)
output_band.SetNoDataValue(no_data_value)

# Save and close the output file
output_dataset.FlushCache()
output_dataset = None

/var/folders/m3/gt5t5g3d2_752qpcpwl512380000gn/T/ipykernel_30379/2176817534.py:23: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir_band + red_band) == 0., 0, (nir_band - red_band) / (nir_band + red_band))


In [5]:
from osgeo import gdal
import numpy as np

def read_geotiff(input_file):
    """Read a GeoTIFF file and return relevant metadata and bands."""
    dataset = gdal.Open(input_file)
    if dataset is None:
        raise FileNotFoundError(f"File {input_file} not found or unable to open.")
    
    x_size = dataset.RasterXSize
    y_size = dataset.RasterYSize
    projection = dataset.GetProjection()
    geotransform = dataset.GetGeoTransform()
    no_data_value = dataset.GetRasterBand(1).GetNoDataValue()

    return dataset, x_size, y_size, projection, geotransform, no_data_value

def write_geotiff(output_path, x_size, y_size, projection, geotransform, no_data_value, data_to_save):
    """Write the processed data to a GeoTIFF file."""
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_path, x_size, y_size, 1, gdal.GDT_Float32)

    output_dataset.SetProjection(projection)
    output_dataset.SetGeoTransform(geotransform)

    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(data_to_save)
    output_band.SetNoDataValue(no_data_value)

    output_dataset.FlushCache()
    output_dataset = None

def calculate_ndvi(input_file, output_file):
    """Main function to process NDVI from input GeoTIFF and save to output file."""
    # assign the returned value of 
    dataset, x_size, y_size, projection, geotransform, no_data_value = read_geotiff(input_file)

    # Read NIR (band 5) and Red (band 4) bands
    nir_band = dataset.GetRasterBand(5).ReadAsArray().astype(np.float32)
    red_band = dataset.GetRasterBand(4).ReadAsArray().astype(np.float32)

    # Calculate NDVI
    ndvi = np.where((nir_band + red_band) == 0., 0, (nir_band - red_band) / (nir_band + red_band))

    # Write NDVI to a new GeoTIFF file
    write_geotiff(output_file, x_size, y_size, projection, geotransform, no_data_value, ndvi)

# Define input and output file paths
input_file = "Stacked_8bands_1.TIF"
output_file = "Stacked_8bands_1_ndvi.TIF"

# Run the NDVI processing
calculate_ndvi(input_file, output_file)

/var/folders/m3/gt5t5g3d2_752qpcpwl512380000gn/T/ipykernel_30379/2127068658.py:43: RuntimeWarning: invalid value encountered in divide
  ndvi = np.where((nir_band + red_band) == 0., 0, (nir_band - red_band) / (nir_band + red_band))


## Assignment 1
Use loops to calculate SAVI and NDSI for the two Landsat-5 images (bands are seperated for each image) I uploaded. 

Requirement: 

Unzip the two image folders under the same root folder, the script should be able to read in the necessary image files  iteratively and aotomatically calculate spectral indices and save the result to a new location. All files should be saved under a new folder, and each file should be named in this format: "NameOfImage_SAVI.tif". For example, for image file LT50290372010217EDC00, the saved name should be "LT50290372010217EDC00_SAVI.tif", but this string (LT50290372010217EDC00) should be automatically extracted.

Make a assignment 5 folder in you git repo and pus it to github. Display the results in QGIS and upload screen shots in PDF or word.

***Hint:***  
Put the calculation of SAVI and NDSI into one function, this function should return the value of SAVI and use a loop to save the results


## DN matching
Let's review the steps for DN matching:
1. find common stable dark and stable bright areas between the two dates that is constant in all bands you are interested in. 
2. Clip those areas from the images. You can do this in [QGIS](https://www.youtube.com/watch?v=hPSIW1W3XjY). In this lab, try to set the clip extent to be a rectangel so that we won't need to worry about no-data value surrounding it.
3. Calculate the average value of the dark and bright areas from both dates for each interested band.
4. for each band interested, calculate the linear relashinship between date 1 and date 2 through the two known points: (average constant dark region DN at date 1, ...date 2), and (average constant bright region DN at date 1, ...date 2). The liner regression is y = ax + b, and now you have values of x and y, calculate a and b based on the two points.
5. For each interested band, correct the DN values of date 2 to date 1 using:  New_DN_date_2 = a * DN_date_2 + b



In [ ]:
# Assignment 1: Automated SAVI and NDSI Calculation
# This script processes two Landsat-5 images and calculates SAVI and NDSI spectral indices

from osgeo import gdal
import numpy as np
import os
import glob

def read_geotiff_band(file_path, band_number=1):
    """Read a specific band from a GeoTIFF file and return array and metadata."""
    dataset = gdal.Open(file_path)
    if dataset is None:
        raise FileNotFoundError(f"File {file_path} not found or unable to open.")
    
    x_size = dataset.RasterXSize
    y_size = dataset.RasterYSize
    projection = dataset.GetProjection()
    geotransform = dataset.GetGeoTransform()
    no_data_value = dataset.GetRasterBand(band_number).GetNoDataValue()
    
    # Read the band as numpy array
    band_array = dataset.GetRasterBand(band_number).ReadAsArray().astype(np.float32)
    
    dataset = None  # Close the dataset
    return band_array, x_size, y_size, projection, geotransform, no_data_value

def write_geotiff(output_path, x_size, y_size, projection, geotransform, no_data_value, data_to_save):
    """Write the processed data to a GeoTIFF file."""
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_path, x_size, y_size, 1, gdal.GDT_Float32)

    output_dataset.SetProjection(projection)
    output_dataset.SetGeoTransform(geotransform)

    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(data_to_save)
    output_band.SetNoDataValue(no_data_value)

    output_dataset.FlushCache()
    output_dataset = None

def calculate_savi(nir_band, red_band, l=0.5):
    """
    Calculate Soil Adjusted Vegetation Index (SAVI)
    SAVI = ((NIR - Red) / (NIR + Red + L)) * (1 + L)
    where L is the soil brightness correction factor (typically 0.5)
    """
    # Handle division by zero
    denominator = nir_band + red_band + l
    savi = np.where(denominator == 0., 0, ((nir_band - red_band) / denominator) * (1 + l))
    return savi

def calculate_ndsi(green_band, swir_band):
    """
    Calculate Normalized Difference Snow Index (NDSI)
    NDSI = (Green - SWIR) / (Green + SWIR)
    """
    # Handle division by zero
    denominator = green_band + swir_band
    ndsi = np.where(denominator == 0., 0, (green_band - swir_band) / denominator)
    return ndsi

def process_landsat_image(image_folder, output_folder):
    """
    Process a single Landsat image to calculate SAVI and NDSI.
    Returns the image name for file naming.
    """
    # Extract image name from folder path
    image_name = os.path.basename(image_folder.rstrip('/'))
    
    # Define band file paths (Landsat-5 bands)
    red_path = os.path.join(image_folder, f"{image_name}_B3.TIF")      # Band 3 (Red)
    nir_path = os.path.join(image_folder, f"{image_name}_B4.TIF")       # Band 4 (NIR)
    green_path = os.path.join(image_folder, f"{image_name}_B2.TIF")     # Band 2 (Green)
    swir_path = os.path.join(image_folder, f"{image_name}_B5.TIF")      # Band 5 (SWIR)
    
    print(f"Processing {image_name}...")
    
    # Read bands and get metadata
    red_band, x_size, y_size, projection, geotransform, no_data_value = read_geotiff_band(red_path)
    nir_band, _, _, _, _, _ = read_geotiff_band(nir_path)
    green_band, _, _, _, _, _ = read_geotiff_band(green_path)
    swir_band, _, _, _, _, _ = read_geotiff_band(swir_path)
    
    # Calculate SAVI
    savi = calculate_savi(nir_band, red_band)
    
    # Calculate NDSI
    ndsi = calculate_ndsi(green_band, swir_band)
    
    # Save SAVI
    savi_output_path = os.path.join(output_folder, f"{image_name}_SAVI.TIF")
    write_geotiff(savi_output_path, x_size, y_size, projection, geotransform, no_data_value, savi)
    print(f"  SAVI saved: {savi_output_path}")
    
    # Save NDSI
    ndsi_output_path = os.path.join(output_folder, f"{image_name}_NDSI.TIF")
    write_geotiff(ndsi_output_path, x_size, y_size, projection, geotransform, no_data_value, ndsi)
    print(f"  NDSI saved: {ndsi_output_path}")
    
    return image_name

# Main processing
def main():
    # Define paths
    current_dir = os.getcwd()
    output_folder = os.path.join(current_dir, "assignment_5_output")
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Define the two Landsat image folders
    image_folders = [
        "LT50290372010217EDC00",
        "LT50290372010233EDC00"
    ]
    
    print("Starting Assignment 1: SAVI and NDSI Calculation")
    print("=" * 50)
    
    # Process each image
    for folder in image_folders:
        if os.path.exists(folder):
            try:
                image_name = process_landsat_image(folder, output_folder)
                print(f"✓ Successfully processed {image_name}")
            except Exception as e:
                print(f"✗ Error processing {folder}: {str(e)}")
        else:
            print(f"✗ Folder {folder} not found")
    
    print("=" * 50)
    print("Assignment 1 processing complete!")
    print(f"Results saved in: {output_folder}")

# Run the main function
if __name__ == "__main__":
    main()


## Assignment 2
Perform DN matching to NIR, SWIR1 and Red bands with the two images I uploaded. Use the image with less cloud as the reference. Save the file after correction. Show on map a comparison between pre-correction and post-correction, compare them to the reference image. Also display where you chose to be the stable dark region and stable bright region. Take screen shots and push your code to github.

Requirements: 
The script need to be as automated as possible, and modularize your code. Points will be deducted if the code is not automated enough. For example, you are reading in files automatilly but you write seperate code for each band to calculate the linear relashionship or you did not wrap linear regression calculation, read & write img files in a function.

***Hint:***
1. Stack the bands before clipping 
2. calculating average: npArray.mean()
3. linear regression calculation is:   
    a (slope) = (y2 - y1) / (x2 - x1)  
    b (intercept) = y1 - a * x1

In [7]:
# Assignment 2: DN Matching for Radiometric Correction
# This script performs DN matching between two Landsat-5 images using stable areas

from osgeo import gdal
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def read_geotiff_band(file_path, band_number=1):
    """Read a specific band from a GeoTIFF file and return array and metadata."""
    dataset = gdal.Open(file_path)
    if dataset is None:
        raise FileNotFoundError(f"File {file_path} not found or unable to open.")
    
    x_size = dataset.RasterXSize
    y_size = dataset.RasterYSize
    projection = dataset.GetProjection()
    geotransform = dataset.GetGeoTransform()
    no_data_value = dataset.GetRasterBand(band_number).GetNoDataValue()
    
    # Read the band as numpy array
    band_array = dataset.GetRasterBand(band_number).ReadAsArray().astype(np.float32)
    
    dataset = None  # Close the dataset
    return band_array, x_size, y_size, projection, geotransform, no_data_value

def write_geotiff(output_path, x_size, y_size, projection, geotransform, no_data_value, data_to_save):
    """Write the processed data to a GeoTIFF file."""
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_path, x_size, y_size, 1, gdal.GDT_Float32)

    output_dataset.SetProjection(projection)
    output_dataset.SetGeoTransform(geotransform)

    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(data_to_save)
    
    # Only set NoData value if it's valid (not None)
    if no_data_value is not None:
        output_band.SetNoDataValue(no_data_value)

    output_dataset.FlushCache()
    output_dataset = None

def stack_bands(image_name, bands_to_stack):
    """
    Stack multiple bands into a single multi-band GeoTIFF file.
    bands_to_stack: list of band numbers to stack (e.g., [3, 4, 5] for Red, NIR, SWIR1)
    """
    print(f"Stacking bands for {image_name}...")
    
    # Read first band to get metadata
    first_band_path = f"{image_name}_B{bands_to_stack[0]}.TIF"
    first_band, x_size, y_size, projection, geotransform, no_data_value = read_geotiff_band(first_band_path)
    
    # Create stacked array
    stacked_array = np.zeros((len(bands_to_stack), x_size, y_size), dtype=np.float32)
    stacked_array[0] = first_band
    
    # Read remaining bands
    for i, band_num in enumerate(bands_to_stack[1:], 1):
        band_path = f"{image_name}_B{band_num}.TIF"
        band_array, _, _, _, _, _ = read_geotiff_band(band_path)
        stacked_array[i] = band_array
    
    # Save stacked image
    output_path = f"{image_name}_stacked.tif"
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_path, x_size, y_size, len(bands_to_stack), gdal.GDT_Float32)
    
    output_dataset.SetProjection(projection)
    output_dataset.SetGeoTransform(geotransform)
    
    for i in range(len(bands_to_stack)):
        output_band = output_dataset.GetRasterBand(i + 1)
        output_band.WriteArray(stacked_array[i])
        if no_data_value is not None:
            output_band.SetNoDataValue(no_data_value)
    
    output_dataset.FlushCache()
    output_dataset = None
    
    print(f"  Stacked image saved: {output_path}")
    return output_path, stacked_array, x_size, y_size, projection, geotransform, no_data_value

def clip_stable_areas(image_path, dark_coords, bright_coords, output_folder):
    """
    Clip stable dark and bright areas from the stacked image.
    dark_coords: (x1, y1, x2, y2) for dark area
    bright_coords: (x1, y1, x2, y2) for bright area
    """
    print("Clipping stable areas...")
    
    # Open the stacked image
    dataset = gdal.Open(image_path)
    if dataset is None:
        raise FileNotFoundError(f"Cannot open {image_path}")
    
    # Get image dimensions
    x_size = dataset.RasterXSize
    y_size = dataset.RasterYSize
    num_bands = dataset.RasterCount
    
    # Read all bands
    bands_data = []
    for i in range(1, num_bands + 1):
        band_data = dataset.GetRasterBand(i).ReadAsArray()
        bands_data.append(band_data)
    
    # Clip dark area
    dark_x1, dark_y1, dark_x2, dark_y2 = dark_coords
    dark_clip = []
    for band_data in bands_data:
        dark_band = band_data[dark_y1:dark_y2, dark_x1:dark_x2]
        dark_clip.append(dark_band)
    
    # Clip bright area
    bright_x1, bright_y1, bright_x2, bright_y2 = bright_coords
    bright_clip = []
    for band_data in bands_data:
        bright_band = band_data[bright_y1:bright_y2, bright_x1:bright_x2]
        bright_clip.append(bright_band)
    
    dataset = None
    
    print(f"  Dark area clipped: {dark_x1},{dark_y1} to {dark_x2},{dark_y2}")
    print(f"  Bright area clipped: {bright_x1},{bright_y1} to {bright_x2},{bright_y2}")
    
    return dark_clip, bright_clip

def calculate_linear_regression(dark_avg_ref, dark_avg_target, bright_avg_ref, bright_avg_target):
    """
    Calculate linear regression parameters (slope and intercept) for DN correction.
    Returns: slope (a) and intercept (b) for equation: y = ax + b
    """
    # Calculate slope: a = (y2 - y1) / (x2 - x1)
    slope = (bright_avg_target - dark_avg_target) / (bright_avg_ref - dark_avg_ref)
    
    # Calculate intercept: b = y1 - a * x1
    intercept = dark_avg_target - slope * dark_avg_ref
    
    return slope, intercept

def apply_dn_correction(band_data, slope, intercept):
    """
    Apply DN correction using linear regression parameters.
    New_DN = slope * Original_DN + intercept
    """
    corrected_band = slope * band_data + intercept
    return corrected_band

def process_dn_matching():
    """
    Main function to process DN matching between two Landsat images.
    """
    print("Starting Assignment 2: DN Matching for Radiometric Correction")
    print("=" * 60)
    
    # Define image names and bands of interest
    image_names = ["LT50290372010217EDC00", "LT50290372010233EDC00"]
    bands_of_interest = [3, 4, 5]  # Red, NIR, SWIR1
    band_names = ["Red", "NIR", "SWIR1"]
    
    # Create output folder
    output_folder = "assignment_2_output"
    os.makedirs(output_folder, exist_ok=True)
    
    # Step 1: Stack bands for both images
    print("\nStep 1: Stacking bands...")
    stacked_paths = []
    stacked_data = []
    
    for image_name in image_names:
        stacked_path, stacked_array, x_size, y_size, projection, geotransform, no_data_value = stack_bands(
            image_name, bands_of_interest
        )
        stacked_paths.append(stacked_path)
        stacked_data.append(stacked_array)
    
    # Step 2: Define stable areas (you'll need to adjust these coordinates based on your images)
    # These are example coordinates - you should identify actual stable areas in your images
    print("\nStep 2: Defining stable areas...")
    
    # Dark area coordinates (adjust based on your image analysis)
    dark_coords = (100, 100, 200, 200)  # (x1, y1, x2, y2)
    
    # Bright area coordinates (adjust based on your image analysis)  
    bright_coords = (300, 300, 400, 400)  # (x1, y1, x2, y2)
    
    print(f"  Dark area: {dark_coords}")
    print(f"  Bright area: {bright_coords}")
    
    # Step 3: Clip stable areas from both images
    print("\nStep 3: Clipping stable areas...")
    dark_areas = []
    bright_areas = []
    
    for i, stacked_path in enumerate(stacked_paths):
        dark_clip, bright_clip = clip_stable_areas(stacked_path, dark_coords, bright_coords, output_folder)
        dark_areas.append(dark_clip)
        bright_areas.append(bright_clip)
    
    # Step 4: Calculate average DN values for each band
    print("\nStep 4: Calculating average DN values...")
    
    # Reference image (less cloudy - you may need to determine which is less cloudy)
    ref_idx = 0  # Assuming first image is reference
    target_idx = 1
    
    print(f"  Reference image: {image_names[ref_idx]}")
    print(f"  Target image: {image_names[target_idx]}")
    
    # Calculate averages for reference image
    ref_dark_avgs = [np.mean(dark_areas[ref_idx][i]) for i in range(len(bands_of_interest))]
    ref_bright_avgs = [np.mean(bright_areas[ref_idx][i]) for i in range(len(bands_of_interest))]
    
    # Calculate averages for target image
    target_dark_avgs = [np.mean(dark_areas[target_idx][i]) for i in range(len(bands_of_interest))]
    target_bright_avgs = [np.mean(bright_areas[target_idx][i]) for i in range(len(bands_of_interest))]
    
    print("  Average DN values:")
    for i, band_name in enumerate(band_names):
        print(f"    {band_name} Band:")
        print(f"      Reference - Dark: {ref_dark_avgs[i]:.2f}, Bright: {ref_bright_avgs[i]:.2f}")
        print(f"      Target - Dark: {target_dark_avgs[i]:.2f}, Bright: {target_bright_avgs[i]:.2f}")
    
    # Step 5: Calculate linear regression parameters for each band
    print("\nStep 5: Calculating linear regression parameters...")
    slopes = []
    intercepts = []
    
    for i, band_name in enumerate(band_names):
        slope, intercept = calculate_linear_regression(
            ref_dark_avgs[i], target_dark_avgs[i],
            ref_bright_avgs[i], target_bright_avgs[i]
        )
        slopes.append(slope)
        intercepts.append(intercept)
        print(f"  {band_name} Band: slope={slope:.4f}, intercept={intercept:.2f}")
    
    # Step 6: Apply DN correction to target image
    print("\nStep 6: Applying DN correction...")
    
    # Read target image bands
    target_bands = []
    for band_num in bands_of_interest:
        band_path = f"{image_names[target_idx]}_B{band_num}.TIF"
        band_data, _, _, _, _, _ = read_geotiff_band(band_path)
        target_bands.append(band_data)
    
    # Apply correction to each band
    corrected_bands = []
    for i, (band_data, slope, intercept) in enumerate(zip(target_bands, slopes, intercepts)):
        corrected_band = apply_dn_correction(band_data, slope, intercept)
        corrected_bands.append(corrected_band)
        
        # Save corrected band
        output_path = os.path.join(output_folder, f"{image_names[target_idx]}_corrected_{band_names[i]}.TIF")
        write_geotiff(output_path, x_size, y_size, projection, geotransform, no_data_value, corrected_band)
        print(f"  Corrected {band_names[i]} band saved: {output_path}")
    
    # Step 7: Create comparison visualization
    print("\nStep 7: Creating comparison visualization...")
    
    # Create a simple comparison plot
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('DN Matching Results Comparison', fontsize=16)
    
    for i, band_name in enumerate(band_names):
        # Original target band
        axes[0, i].imshow(target_bands[i], cmap='viridis')
        axes[0, i].set_title(f'Original {band_name}')
        axes[0, i].axis('off')
        
        # Corrected target band
        axes[1, i].imshow(corrected_bands[i], cmap='viridis')
        axes[1, i].set_title(f'Corrected {band_name}')
        axes[1, i].axis('off')
    
    # Add rectangles to show stable areas
    for ax in axes.flat:
        # Dark area rectangle
        dark_rect = Rectangle((dark_coords[0], dark_coords[1]), 
                             dark_coords[2]-dark_coords[0], 
                             dark_coords[3]-dark_coords[1],
                             linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(dark_rect)
        
        # Bright area rectangle
        bright_rect = Rectangle((bright_coords[0], bright_coords[1]), 
                               bright_coords[2]-bright_coords[0], 
                               bright_coords[3]-bright_coords[1],
                               linewidth=2, edgecolor='blue', facecolor='none')
        ax.add_patch(bright_rect)
    
    plt.tight_layout()
    comparison_path = os.path.join(output_folder, 'dn_matching_comparison.png')
    plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"  Comparison plot saved: {comparison_path}")
    
    print("\n" + "=" * 60)
    print("Assignment 2: DN Matching Complete!")
    print(f"Results saved in: {output_folder}")
    print("\nNext steps:")
    print("1. Review the stable area coordinates and adjust if needed")
    print("2. Open corrected images in QGIS for detailed analysis")
    print("3. Take screenshots of comparison maps")
    print("4. Push code and results to GitHub")

# Run the DN matching process
if __name__ == "__main__":
    process_dn_matching()


Starting Assignment 2: DN Matching for Radiometric Correction

Step 1: Stacking bands...
Stacking bands for LT50290372010217EDC00...


ValueError: could not broadcast input array from shape (7161,8031) into shape (8031,7161)